In [108]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import time

In [49]:
df = pd.read_excel("Demo.xlsx")
df['Date Of Joining'] = pd.to_datetime(df['Date Of Joining']).dt.strftime('%Y-%m-%d')


In [3]:
driver = webdriver.Edge()
driver.get('https://printyourcopy.com/free-payslip-generator')


In [50]:
df.head()

,T. Calenders Days,Month,Full Name (As Per CNIC),CNIC Number,Designation,Date Of Joining,Gross Salary,Bonus / Commission,Increment,Reimbursment Amount,Compensation,Adjustments,Absents Deduction,Lates Deduction,Other Deductions,Payroll Tax Deduction,Net Salary Payable\n(in PKR),Total Deductions\n(Amount),Salary Payable\n(Amount)
0,31,OCTOBER' 2024,Muhammad Haris Dilshad,42000-123456-7,Accounts - Assistant Manager,2024-01-26,5,2,1,3,8,10,12,15,0,800,1200,400,800
1,31,OCTOBER' 2024,Fahad Bin Zahid,42000-123456-8,Accounts - Executive,2024-01-27,5,2,1,3,8,10,0,0,0,800,1200,400,800
2,31,OCTOBER' 2024,Ibrahim Najmuddin,42000-123456-9,Accounts - Executive,2024-01-28,5,2,1,3,8,10,0,0,0,800,1200,800,400
3,31,OCTOBER' 2024,Zuhaib Ghori,42000-123456-10,Accounts - Executive,2024-01-29,5,2,1,3,8,10,0,0,0,800,1200,800,400
4,31,OCTOBER' 2024,Muhammad Sabih Uddin Farooqi,42000-123456-11,Accounts - HOD,2024-01-30,5,2,1,3,8,10,0,0,0,800,1200,400,800


In [ ]:
# for i in range(3):
#     value = df.loc[i, 'Full Name (As Per CNIC)']
#     print(value)

Muhammad Haris Dilshad
Fahad Bin Zahid
Ibrahim Najmuddin


In [114]:
#Logo
logo = os.path.abspath("Logo.jpg")
file_input = driver.find_element(By.XPATH, '//*[@id="logo"]').send_keys(logo)

#payslip no.
driver.find_element(By.XPATH, '//*[@id="serial"]').clear()

# Employee Details
employee_details= f"""{df.loc[1, 'Full Name (As Per CNIC)']}
{df.loc[1, 'CNIC Number']}
{df.loc[1, 'Designation']}
DOJ: {df.loc[1, 'Date Of Joining']}
"""
driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(employee_details)

#Employer Details
employer_details = f"""
Robust Support & Solutions
Office No.404A, Fortune Tower
PECHS Block 6, Karachi, Pakistan
Phone & WhatsApp: 0311-3859635
"""
driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(employer_details)

#Notimportant
driver.find_element(By.XPATH, '//*[@id="employeeSign"]').clear()
driver.find_element(By.XPATH, '//*[@id="employerSign"]').clear()

#Currency
driver.find_element(By.XPATH, '//*[@id="currency"]').clear()
driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys("PKR")

time.sleep(2)
# Earning boxes
n = 4
for i in range(4):
    wait = WebDriverWait(driver,5)
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button')))
    button.click()    
    n += 1


In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Basic pay clearing
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').clear()
time.sleep(0.2)

# Bonus / Commission clearing 
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').clear()
time.sleep(0.2)

# Reimbursement Amount clearing
driver.find_element(By.XPATH, '//*[@id="earnings.2.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').clear()
time.sleep(0.2)

# Increment clearing
driver.find_element(By.XPATH, '//*[@id="earnings.3.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').clear()
time.sleep(0.2)

# Compensation
driver.find_element(By.XPATH, '//*[@id="earnings.4.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').clear()
time.sleep(0.2)

# Adjustments
driver.find_element(By.XPATH, '//*[@id="earnings.5.description"]').clear()
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').clear()
time.sleep(0.2) 

In [ ]:
# Basic pay values
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys("Gross Salary")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal delay

# Bonus / Commission values
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys("Bonus / Commission")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal delay

# Reimbursement Amount values
driver.find_element(By.XPATH, '//*[@id="earnings.2.description"]').send_keys("Reimbursment Amount")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal delay


# Basic pay values
driver.find_element(By.XPATH, '//*[@id="earnings.0.description"]').send_keys("Gross Salary")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal delay

# Bonus / Commission values
driver.find_element(By.XPATH, '//*[@id="earnings.1.description"]').send_keys("Bonus / Commission")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal delay

# Reimbursement Amount values
driver.find_element(By.XPATH, '//*[@id="earnings.2.description"]').send_keys("Reimbursment Amount")
time.sleep(0.2)  # Minimal delay
driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys("56")
time.sleep(0.2)  # Minimal dela*y

In [41]:
df.columns

Index(['T. Calenders Days', 'Month', 'Full Name (As Per CNIC)', 'CNIC Number',
       'Designation', 'Date Of Joining', 'Gross Salary', 'Bonus / Commission',
       'Increment', 'Reimbursment Amount', 'Compensation', 'Adjustments',
       'Absents Deduction', 'Lates Deduction', 'Other Deductions',
       'Payroll Tax Deduction', 'Net Salary Payable\n(in PKR)',
       'Total Deductions\n(Amount)', 'Salary Payable\n(Amount)'],
      dtype='object')

KeyboardInterrupt: 